In [1]:
import numpy as np
from copy import deepcopy
import itertools
import matplotlib
from numpy import radians as rad
from matplotlib.animation import FuncAnimation
from scipy.ndimage import convolve,convolve1d
import cmasher
import seaborn as sns
from collections import defaultdict
from manim import *
import networkx as nx
from scipy.interpolate import interp1d
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import random
from networkx.drawing.nx_pydot import graphviz_layout
from scipy.special import softmax
import itertools as it
from collections import deque
import torch.nn.functional as F
import pandas as pd
from scipy.integrate import quad

# Derangements with binomial distribution

In [3]:
# A) Pakkeleg - introducing the problem. First the characters and the presents, they take the presents. 
# They shuffled a few times, then the derangement and the problem.
# B) Binomial distribution (introduce briefly). 
# Reformulate as: each of the n elements has a 1/n chance of being in its original position.
# For the event "no element remains in its original position," 
# we can model this as the complement of the binomial distribution with k = 0 and p = 1/n. Getting the formula
# C) Approximate with e. And then notice that it is Poisson with k = 0! 
# "As the matter of fact, a binomial distribution is well approximated by poisson distribution in 
# cases when number of successes is much smaller than n - it doesn't even always have to be zero!
# So, alternatively, we could skip the binomial distribution altogether, saying: 
# what is the probability of 0 events happening in a situation where 1 event is expected to happen?
# which corresponds to a Poisson distributions with parameter lambda equal 1"

In [40]:
class Intro(Scene):
    
    def construct(self):
        # Create viking images
        viking_yellow = ImageMobject("images/yellow_viking.png").scale(0.5)
        viking_blue = ImageMobject("images/blue_viking.png").scale(0.5)
        viking_red = ImageMobject("images/red_viking.png").scale(0.5)
        viking_green = ImageMobject("images/green_viking.png").scale(0.5)

        viking_yellow.shift(LEFT * 3)
        viking_blue.next_to(viking_yellow, RIGHT, buff=1)
        viking_red.next_to(viking_blue, RIGHT, buff=1)
        viking_green.next_to(viking_red, RIGHT, buff=1)
        
        vikings = Group(viking_yellow, viking_blue, viking_red, viking_green).move_to(UP)

        present_yellow = ImageMobject("images/yellow_present.png").scale(0.3)
        present_blue = ImageMobject("images/blue_present.png").scale(0.3)
        present_red = ImageMobject("images/red_present.png").scale(0.3)
        present_green = ImageMobject("images/green_present.png").scale(0.3)

        direction = DOWN
        present_yellow.next_to(viking_yellow, direction)
        present_blue.next_to(viking_blue, direction)
        present_red.next_to(viking_red, direction)
        present_green.next_to(viking_green, direction)
        
        presents = Group(present_yellow, present_blue, present_red, present_green)

        # Animations
        self.play(FadeIn(vikings))  # Show vikings together
        self.play(FadeIn(presents))  # Show presents together, positioned next to vikings

        # Pause to display final arrangement
        self.wait(1)


In [41]:
%manim Intro

Manim Community v0.18.1

[01/11/25 15:26:23] INFO     Animation 0 : Using cached data (hash :                           ]8;id=842002;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=874553;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             3977891868_285927764_223132457)                                                       

[01/11/25 15:26:25] INFO     Animation 1 : Partial movie file written in                   ]8;id=154726;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=406929;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/partial_movie_files/Intro/2852726489_12799579                         
                             77_3867173521.mp4'                                                                    

[01/11/25 15:26:26] INFO     Animation 2 : Partial movie file written in                   ]8;id=26196;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=131953;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/partial_movie_files/Intro/2852726489_29193400                         
                             33_454327810.mp4'                                                                     

                    INFO     Combining to Movie file.                                      ]8;id=579908;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=815571;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#617\617]8;;\

[01/11/25 15:26:27] INFO                                                                   ]8;id=773347;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=122747;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene_file_writer.py#737\737]8;;\
                             File ready at                                                                         
                             '/Users/nmb127/Documents/animations/notebooks/media/videos/no                         
                             tebooks/1080p60/Intro.mp4'                                                            
                                                                                                                   

                    INFO     Rendered Intro                                                            ]8;id=57375;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=409239;file:///Users/nmb127/opt/anaconda3/envs/youtube/lib/python3.9/site-packages/manim/scene/scene.py#247\247]8;;\
                             Played 3 animations                                                                   